### Установка пакетов

In [295]:
!pip install pyspark

### Импорт библиотек

In [296]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, trim, regexp_replace, desc
from operator import add
import os

### Создание экземпляра **SparkSession**

In [297]:
spark = SparkSession.builder.appName('Homework_1_lab3').getOrCreate()

### Генерация текстового файла

In [298]:
if not os.path.exists('data'):
  os.makedirs('data')

In [299]:
with open('data/words.txt', 'w') as f:
  f.writelines([
    'lorem ipsum dolor\n',
    'lorem, ipsum dolor   lorem ipsum dolor\n',
    'lorem ipsum__ dolor lorem ?%@ ipsum dolor lorem ipsum dolor\n',
    'lorem%& ipsum dolor lorem ipsum __dolor lorem ipsum ,dolor lorem ipsum dolor lorem dolorem'
  ])

### Считывание файла

In [300]:
text = spark.read.text('data/words.txt')

In [301]:
text.show()

+--------------------+
|               value|
+--------------------+
|   lorem ipsum dolor|
|lorem, ipsum dolo...|
|lorem ipsum__ dol...|
|lorem%& ipsum dol...|
+--------------------+



### Преобразование датафрейма

In [302]:
def prepare_text(col):
  return explode(split(lower(trim(regexp_replace(col, '[^A-Za-z0-9]', ' '))) , ' '))

words = text.select(prepare_text(text.value).alias('word')).where('word != ""')

In [303]:
words.show()

+-----+
| word|
+-----+
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
+-----+
only showing top 20 rows



### Подсчет количество каждого слова в тексте

In [304]:
words_count = words.groupBy('word').count()

In [305]:
words_count.show()

+-------+-----+
|   word|count|
+-------+-----+
|  lorem|   11|
|  dolor|   10|
|  ipsum|   10|
|dolorem|    1|
+-------+-----+



### Сортировка слов в порядке убывания их количества в тексте

In [306]:
sorted_words_count = words_count.orderBy(desc('count'))

In [307]:
sorted_words_count.show()

+-------+-----+
|   word|count|
+-------+-----+
|  lorem|   11|
|  dolor|   10|
|  ipsum|   10|
|dolorem|    1|
+-------+-----+



### Получение самого часто встречаемого слова

In [308]:
sorted_words_count.first()

Row(word='lorem', count=11)

### Получение самого редко встречаемого слова

In [309]:
sorted_words_count.tail(1)

[Row(word='dolorem', count=1)]

### Подсчет средней встречаемости слов в тексте

In [310]:
sorted_words_count.groupBy().mean().collect()

[Row(avg(count)=8.0)]

### Подсчет уникального числа слов в тексте


In [311]:
sorted_words_count.count()

4

### Подсчет общего числа слов в тексте

In [312]:
sorted_words_count.agg({'count': 'sum'}).collect()[0][0]

32